In [1]:
# Cell 1: Imports and Constants

import math, heapq
from dataclasses import dataclass
import random

#CONSTANTS
MAX_WALK_MIN = 25 #maximum walking amount of 25 minutes
PRICE_WEIGHT = 2.0 #Every dollar for the parking lot price = 2 minutes

In [2]:
# Cell 2: Graph Definition and Dijkstra's Algorithm

# Nodes for intersections
NODES = ["S","A","B","C","E","D"]  # S = start, D = final destination

# Directed edges with travel time (minutes)
TRAVEL_TIMES = {
    ("S","A"): 4,
    ("S","B"): 6,
    ("A","C"): 5,
    ("B","C"): 2,
    ("B","E"): 4,
    ("C","D"): 6,
    ("C","E"): 3,
    ("E","D"): 2,
    ("A","E"): 8
}

#An adjacency list that acts as a map that shows which nodes are directly reachable from each node based off the graph.
ADJ = {n: [] for n in NODES}
for (u,v) in TRAVEL_TIMES:
    ADJ[u].append(v)

#Dijkstra's Algorithm to finds the shortest driving time from the starting node to all other nodes
def dijkstra(start):

    #Store shortest distance from start "S" to each node
    distance = {node: math.inf for node in NODES}
    #Stores the previous node used to reach each node
    previous = {node: None for node in NODES}
    distance[start] = 0
    queue = [(0, start)]

    #Repeatedly select the closest unexplored node and update shorter paths to its neighbors
    while queue:

        current_distance, current_node = heapq.heappop(queue)
        if current_distance > distance[current_node]:
            continue

        for neighbor in ADJ[current_node]:
            new_distance = current_distance + TRAVEL_TIMES[(current_node, neighbor)]
            if new_distance < distance[neighbor]:
                distance[neighbor] = new_distance
                previous[neighbor] = current_node
                heapq.heappush(queue, (new_distance, neighbor))

    return distance, previous

#Builds and returns the shortest path from the start node to the target by following the stored previous node links
def path(previous, target):
    route = []
    current = target

    while current is not None:
        route.append(current)
        current = previous[current]

    route.reverse()
    return route


In [3]:
# Cell 3: Parking Lot Definition and Cost Function

# Parking lot information
@dataclass(frozen=True)
class Lot:
    name: str
    node: str
    price: float           # Price in dollars
    walk_min: float        # Walking time in minutes
    capacity: int
    occupied: int

    # Function to check if parking lot has spots available
    @property
    def available(self):
        return self.occupied < self.capacity
    
# Function that will return the total cost
def total_cost(drive_min, lot):
    if not lot.available: return math.inf
    if lot.walk_min > MAX_WALK_MIN: return math.inf
    return drive_min + lot.walk_min + PRICE_WEIGHT*lot.price

In [4]:
# Cell 4: Hill Climbing Algorithm for Parking Lot Selection

# Hill climbing algorithm to find the best parking lot based on the total cost function
# It starts with a random valid lot and iteratively moves to a better neighboring lot until no improvement is found or max iterations are reached.
def hill_climb_parking_lot(start_node, lots, max_iters=50, seed=42):

    random.seed(seed)

    # Find shortest driving times from start to all parking lots
    dist, prev = dijkstra(start_node)

    # Calculate total cost for each lot based on driving time, walking time, and lot price
    cost = {}
    for lot in lots:
        drive = dist.get(lot.node, math.inf)
        cost[lot] = total_cost(drive, lot)

    # Filter out lots that are invalid 
    valid_lots = [lot for lot in lots if not math.isinf(cost[lot])]
    if not valid_lots:
        raise ValueError("No valid parking lots available (all full/too far/unreachable).")

    # Start by randomly selecting a valid lot
    current = random.choice(valid_lots)

    # Print initial state
    print(f"Initial lot: {current.name} | cost={cost[current]:.2f}")

    # Hill climbing loop
    for i in range(1, max_iters + 1):
        # Find neighbors (other valid lots)
        neighbors = [lot for lot in lots if lot != current]

        # Filter neighbors to only those that are valid and have a defined cost
        neighbors = [lot for lot in neighbors if not math.isinf(cost[lot])]
        if not neighbors:
            print(f"Stop at iteration {i:02d}: no valid neighbors available.")
            break

        # Choose the neighbor with the lowest cost
        best_neighbor = min(neighbors, key=lambda l: cost[l])
        if cost[best_neighbor] < cost[current]:
            current = best_neighbor
            print(f"Iteration {i:02d}: move -> {current.name} | cost={cost[current]:.2f}")
        else:
            # No better neighbor found, stop climbing
            print(f"Stop at iteration {i:02d}: no better neighbor found.")
            break
    
    # Final result 
    return current, cost, dist, prev


In [6]:
# Cell 5: Main Execution - Calculate and Print Parking Lot Options, Run Hill Climbing

# 4 parking lots for testing
LOTS = [
    Lot("P1","A", price = 5, walk_min = 15, capacity = 10, occupied = 7),
    Lot("P2","B", price = 3, walk_min = 20, capacity = 10, occupied = 10),
    Lot("P3","C", price = 4, walk_min = 10, capacity = 10, occupied = 2),
    Lot("P4","E", price = 10, walk_min = 5, capacity = 10, occupied = 9),
]

#Runs dijkstra's algorithm starting from node S
dist, prev = dijkstra("S")

#Parking lot table
rows = []
for lot in LOTS:
    drive = dist[lot.node]
    walk = lot.walk_min
    tot  = total_cost(drive, lot)
    pth  = "->".join(path(prev, lot.node))
    rows.append((tot, lot.name, lot.node, pth, drive, walk, lot.price, lot.available))

#Print header for parking lot options
print(f"{'Lot':<3} {'Node':<4} {'Path':<20} {'Drive':>10} {'Walk':>10} {'Price':>9} {'Total':>10} Cost")

#Sorts and prints each parking lots calculated information
for tot, name, node, pth, drive, walk, price, avail in sorted(rows):
    if not avail:
        status = "(FULL)"
    elif walk > MAX_WALK_MIN:
        status = "(Too Far)"
    else:
        status = ""

    tot_str = "N/A" if math.isinf(tot) else f"{tot:.2f}"
    print(f"{name:<3} {node:<4} {pth:<20} {drive:>10.2f} {walk:>10.2f} {price:>9.2f} {tot_str:>10} {status}")

print("\nRun Hill Climbing Algorithm:\n")
# Run the hill climbing algorithm
best_lot, cost_map, dist, prev = hill_climb_parking_lot("S", LOTS, max_iters=50, seed=1)

# Build the best path
best_drive = dist[best_lot.node]
best_path = "->".join(path(prev, best_lot.node))
best_total = cost_map[best_lot]

# Print the best parking lot and its details
print(f"\nChosen lot: {best_lot.name}")
print(f"Route: {best_path}")
print(f"Drive time: {best_drive:.2f} min")
print(f"Walk time: {best_lot.walk_min:.2f} min")
print(f"Price: ${best_lot.price:.2f}")
print(f"Total objective cost: {best_total:.2f}")


Lot Node Path                      Drive       Walk     Price      Total Cost
P3  C    S->B->C                    8.00      10.00      4.00      26.00 
P1  A    S->A                       4.00      15.00      5.00      29.00 
P4  E    S->B->E                   10.00       5.00     10.00      35.00 
P2  B    S->B                       6.00      20.00      3.00        N/A (FULL)

Run Hill Climbing Algorithm:

Initial lot: P1 | cost=29.00
Iteration 01: move -> P3 | cost=26.00
Stop at iteration 02: no better neighbor found.

Chosen lot: P3
Route: S->B->C
Drive time: 8.00 min
Walk time: 10.00 min
Price: $4.00
Total objective cost: 26.00
